# Free surface model generator

**Library imports**

In [1]:
import uuid

import numpy as np

from DataModelDict import DataModelDict as DM

import atomman as am

from pathlib import Path

import iprPy

In [2]:
def vectstr(vect):
    out = ''
    for v in vect:
        out += f'{repr(v)} '
    return out.strip()

def gen_free_surface_model(hkl, uvws, family, atomshift, setting=None):
    if len(hkl) == 3:
        model_id = family + '--' + '%i%i%i' % tuple(hkl)
    elif len(hkl) == 4:
        model_id = family + '--' + '%i%i%i%i' % (np.abs(hkl[0]), np.abs(hkl[1]), np.abs(hkl[2]), np.abs(hkl[3]))
    if setting is not None:
        model_id += '-' + setting
    fname = Path(model_id + '.json')
    
    if fname.is_file():
        #return None
        old = DM(fname.as_posix())
        key = old['free-surface']['key']
    else:
        key = str(uuid.uuid4())
    
    model = DM()
    model['free-surface'] = DM()
    model['free-surface']['key'] = key
    model['free-surface']['id'] = model_id
    model['free-surface']['system-family'] = family
    
    model['free-surface']['calculation-parameter'] = DM()
    if len(hkl) == 3:
        model['free-surface']['calculation-parameter']['a_uvw'] = '%2i %2i %2i' % tuple(uvws[0])
        model['free-surface']['calculation-parameter']['b_uvw'] = '%2i %2i %2i' % tuple(uvws[1])
        model['free-surface']['calculation-parameter']['c_uvw'] = '%2i %2i %2i' % tuple(uvws[2])
    elif len(hkl) == 4:
        model['free-surface']['calculation-parameter']['a_uvw'] = vectstr(uvws[0])
        model['free-surface']['calculation-parameter']['b_uvw'] = vectstr(uvws[1])
        model['free-surface']['calculation-parameter']['c_uvw'] = vectstr(uvws[2])
    model['free-surface']['calculation-parameter']['cutboxvector'] = 'c'
    model['free-surface']['calculation-parameter']['atomshift'] = vectstr(atomshift)
    
    with open(model_id + '.json', 'w') as f:
        model.json(fp=f, indent=4)

In [3]:
database = iprPy.load_database('master')
protos = database.get_records_df(style='crystal_prototype')

# Limit to elemental prototypes
protos = protos[protos.natypes == 1]

In [4]:
np.unique(protos.crystal_family)

array(['cubic', 'hexagonal', 'tetragonal'], dtype=object)

## Cubic

In [5]:
families = protos[protos.crystal_family == 'cubic'].id.tolist()
hkls = [
    [1,0,0],
    [1,1,0],
    [1,1,1],
    [2,1,0],
    [2,1,1],
    [2,2,1],
    [3,1,0],
    [3,1,1],
    [3,2,0],
    [3,2,1],
    [3,2,2],
    [3,3,1],   
    [3,3,2],
       ]

## Hexagonal

In [8]:
families = protos[protos.crystal_family == 'hexagonal'].id.tolist()
hkls = [
    [1,1,-2,0],
    [1,0,-1,0],
    [2,1,-3,0],
    [0,0,0,1],
    [1,1,-2,1],
    [1,0,-1,1],
    [2,0,-2,1],
    [2,1,-3,1],
    [2,2,-4,1],
    [1,0,-1,2],
    [2,-1,-1,2],
    [2,1,-3,2],  
]

## Tetragonal

In [10]:
families = protos[protos.crystal_family == 'tetragonal'].id.tolist()
hkls = [
    [1,0,0],
    [0,0,1],
    [1,0,1],
    [1,1,0],
    [1,1,1],
    [2,0,1],
    [1,1,2],
    [2,1,1],    
]

## Generate

In [11]:
for family in families:
    ucellfile = Path(iprPy.libdir, 'crystal_prototype', family+'.json')
    ucell = am.load('system_model', ucellfile.as_posix())
    
    for hkl in hkls:
        uvws = am.defect.free_surface_basis(hkl, box=ucell.box)
        # Rotate ucell
        system = ucell.rotate(uvws)

        # Get unique scaled z coordinates
        allsz = system.atoms_prop(key='pos', scale=True)[:,2]

        sz = np.array([])
        for z in allsz:

            if np.isclose(z, 1.0) or np.isclose(z, 0.0):
                z = 0.0

            if np.sum(np.isclose(sz, z)) == 0:
                sz = np.append(sz, z)

        sz = np.sort(sz)

        # Wraparound
        atomshifts = [np.array([0.0, 0.0, ((sz[-1] - 1 + sz[0]) / 2)])]
        deltas = [sz[0] - (sz[-1] - 1)]

        # All middle
        for i in range(len(sz)-1):
            deltatest = sz[i+1] - sz[i]
            newdelta = True
            for delta in deltas:
                if np.isclose(delta, deltatest):
                    newdelta = False
                    break
            if newdelta:
                atomshifts.append(np.array([0.0, 0.0, ((sz[i] + sz[i+1]) / 2)]))
                deltas.append(deltatest)
        
        if len(atomshifts) == 1:
            gen_free_surface_model(hkl, uvws, family, atomshifts[0])
        else:
            for i, atomshift in enumerate(atomshifts):
                gen_free_surface_model(hkl, uvws, family, atomshift, setting=str(i+1))